In [ ]:
import os
from google.colab import drive

# --- Mount drive and basic config ---
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/protein-multimodal')

Mounted at /content/drive


In [ ]:
PROJECT_ROOT = "/content/drive/MyDrive/protein-multimodal"
DATA_DIR = f"{PROJECT_ROOT}/data"
RAW_DIR = f"{DATA_DIR}/raw"
PROCESSED_DIR = f"{DATA_DIR}/processed"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

In [ ]:
import json
import re
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

### **Load Raw Data**

In [ ]:
raw_file = f"{RAW_DIR}/proteins_raw_10k.json"

with open(raw_file, "r") as f:
    proteins = json.load(f)

print(f"Loaded {len(proteins)} proteins from {raw_file}")

Loaded 10000 proteins from /content/drive/MyDrive/protein-multimodal/data/raw/proteins_raw_10k.json


### **Dataset Processing**

In [ ]:
def process_dataset(proteins):
    """
    Additional processing and quality control:
    - Valid amino acids only
    - Compute function word count
    """

    processed = []
    valid_aas = set("ACDEFGHIKLMNPQRSTVWY")

    skipped_reasons = {
        "invalid_aas": 0,
        "no_sequence": 0,
        "no_function": 0,
        "too_short_seq": 0,
        "too_short_func": 0,
    }

    for protein in tqdm(proteins, desc="Processing"):
        seq = protein.get("sequence", "")
        func = protein.get("function", "")

        # Must have sequence
        if not seq:
            skipped_reasons["no_sequence"] += 1
            continue

        # Must have function
        if not func:
            skipped_reasons["no_function"] += 1
            continue

        # Check for valid amino acids only
        if not all(aa in valid_aas for aa in seq):
            skipped_reasons["invalid_aas"] += 1
            continue

        # Minimum sequence length (very short sequences might be fragments)
        if len(seq) < 30:
            skipped_reasons["too_short_seq"] += 1
            continue

        # Minimum function description length
        if len(func) < 20:
            skipped_reasons["too_short_func"] += 1
            continue

        # Function length in words
        func_words = len(func.split())
        protein["function_words"] = func_words

        processed.append(protein)

    print(f"\nProcessed {len(processed)} proteins")
    print(f"Filtered out {len(proteins) - len(processed)} proteins:")
    for reason, count in skipped_reasons.items():
        if count > 0:
            print(f"   - {reason}: {count}")

    return processed

processed_proteins = process_dataset(proteins)

Processing:   0%|          | 0/10000 [00:00<?, ?it/s]


Processed 9987 proteins
Filtered out 13 proteins:
   - invalid_aas: 13


### **Dataset Statistics**

In [ ]:
df_processed = pd.DataFrame(processed_proteins)

print(f"\n📊 Dataset Statistics:")
print(f"  - Total proteins: {len(processed_proteins)}")
print(f"  - Unique accession IDs: {df_processed['accession'].nunique()}")

print("\n📏 Sequence Length Distribution:")
print(df_processed["length"].describe())

print("\n📝 Function Word Count Distribution:")
print(df_processed["function_words"].describe())

print("\n🧬 Organism Distribution (Top 5):")
if "organism" in df_processed.columns:
    print(df_processed["organism"].value_counts().head())

print("\n🧪 Sample Protein:")
sample = df_processed.iloc[0]
print(f"  - Accession: {sample['accession']}")
print(f"  - Name: {sample['protein_name']}")
print(f"  - Gene: {sample['gene_name']}")
print(f"  - Organism: {sample['organism']}")
print(f"  - Sequence length: {sample['length']}")
print(f"  - Sequence: {sample['sequence'][:60]}...")
print(f"  - Function: {sample['function'][:200]}...")
print(f"  - Function words: {sample['function_words']}")
print(f"  - GO terms: {len(sample.get('go_terms', []))}")
print(f"  - EC numbers: {sample.get('ec_numbers', [])}")


📊 Dataset Statistics:
  - Total proteins: 9987
  - Unique accession IDs: 9987

📏 Sequence Length Distribution:
count     9987.000000
mean       623.614599
std        690.226931
min         34.000000
25%        291.000000
50%        463.000000
75%        745.000000
max      34350.000000
Name: length, dtype: float64

📝 Function Word Count Distribution:
count    9987.000000
mean       85.209172
std        88.902120
min         2.000000
25%        29.000000
50%        57.000000
75%       112.000000
max      1558.000000
Name: function_words, dtype: float64

🧬 Organism Distribution (Top 5):
organism
Homo sapiens    9987
Name: count, dtype: int64

🧪 Sample Protein:
  - Accession: A0A1B0GTW7
  - Name: Ciliated left-right organizer metallopeptidase
  - Gene: CIROP
  - Organism: Homo sapiens
  - Sequence length: 788
  - Sequence: MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKSRSSSLTLPSSRDPQ...
  - Function: Putative metalloproteinase that plays a role in left-right patterning process...
  - Func

### **Train/Val/Test Splits**

In [ ]:
def create_splits(
    proteins,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42,
):
    """
    Split dataset into train, validation, and test sets

    Args:
        proteins: List of protein dictionaries
        train_ratio: Proportion for training (default 0.8)
        val_ratio: Proportion for validation (default 0.1)
        test_ratio: Proportion for test (default 0.1)
        random_state: Random seed for reproducibility

    Returns:
        train_data, val_data, test_data: Three lists of protein dictionaries
    """
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1.0"

    # First split: separate out training data
    train_data, temp_data = train_test_split(
        proteins,
        test_size=(1 - train_ratio),
        random_state=random_state,
        shuffle=True
    )

    # Second split: divide remaining data into val and test
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_data, test_data = train_test_split(
        temp_data,
        test_size=(1 - val_ratio_adjusted),
        random_state=random_state,
        shuffle=True
    )

    return train_data, val_data, test_data


In [ ]:
train_proteins, val_proteins, test_proteins = create_splits(
    processed_proteins,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42
)

In [ ]:
print("Dataset split created:")
print(f"  - Train: {len(train_proteins)} proteins ({len(train_proteins)/len(processed_proteins)*100:.1f}%)")
print(f"  - Validation: {len(val_proteins)} proteins ({len(val_proteins)/len(processed_proteins)*100:.1f}%)")
print(f"  - Test: {len(test_proteins)} proteins ({len(test_proteins)/len(processed_proteins)*100:.1f}%)")


Dataset split created:
  - Train: 7989 proteins (80.0%)
  - Validation: 999 proteins (10.0%)
  - Test: 999 proteins (10.0%)


### **Save files**

In [ ]:
def save_split(data, filepath):
    """Save split to JSON file"""
    with open(filepath, "w") as f:
        json.dump(data, f, indent=2)

    # Get file size
    size_mb = os.path.getsize(filepath) / (1024 * 1024)
    print(f"Saved {len(data)} proteins to {filepath} ({size_mb:.2f} MB)")


print("\nSaving processed datasets...")
save_split(train_proteins, f"{PROCESSED_DIR}/train.json")
save_split(val_proteins, f"{PROCESSED_DIR}/val.json")
save_split(test_proteins, f"{PROCESSED_DIR}/test.json")


Saving processed datasets...
Saved 7989 proteins to /content/drive/MyDrive/protein-multimodal/data/processed/train.json (26.27 MB)
Saved 999 proteins to /content/drive/MyDrive/protein-multimodal/data/processed/val.json (3.18 MB)
Saved 999 proteins to /content/drive/MyDrive/protein-multimodal/data/processed/test.json (3.06 MB)


In [ ]:
summary = {
    "total_proteins": int(len(processed_proteins)),
    "train_size": int(len(train_proteins)),
    "val_size": int(len(val_proteins)),
    "test_size": int(len(test_proteins)),
    "avg_sequence_length": float(df_processed["length"].mean()),
    "std_sequence_length": float(df_processed["length"].std()),
    "min_sequence_length": int(df_processed["length"].min()),
    "max_sequence_length": int(df_processed["length"].max()),
    "avg_function_words": float(df_processed["function_words"].mean()),
    "std_function_words": float(df_processed["function_words"].std()),
    "min_function_words": int(df_processed["function_words"].min()),
    "max_function_words": int(df_processed["function_words"].max()),
    "unique_accessions": int(df_processed["accession"].nunique()),
}

with open(f"{PROCESSED_DIR}/dataset_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print(f"\nSaved summary to {PROCESSED_DIR}/dataset_summary.json")



Saved summary to /content/drive/MyDrive/protein-multimodal/data/processed/dataset_summary.json


### **Verify Overlap**

In [ ]:
# Load and verify train split
with open(f"{PROCESSED_DIR}/train.json", "r") as f:
    train_check = json.load(f)
    print(f"\n✓ Train split verification: {len(train_check)} proteins loaded")
    print(f"  Sample protein keys: {list(train_check[0].keys())}")

# Load and verify val split
with open(f"{PROCESSED_DIR}/val.json", "r") as f:
    val_check = json.load(f)
    print(f"\n✓ Val split verification: {len(val_check)} proteins loaded")

# Load and verify test split
with open(f"{PROCESSED_DIR}/test.json", "r") as f:
    test_check = json.load(f)
    print(f"\n✓ Test split verification: {len(test_check)} proteins loaded")

# Verify no overlap between splits
train_ids = set(p["accession"] for p in train_check)
val_ids = set(p["accession"] for p in val_check)
test_ids = set(p["accession"] for p in test_check)

overlap_train_val = train_ids & val_ids
overlap_train_test = train_ids & test_ids
overlap_val_test = val_ids & test_ids

print(f"\n🔍 Checking for overlaps between splits:")
print(f"  - Train ∩ Val: {len(overlap_train_val)} proteins")
print(f"  - Train ∩ Test: {len(overlap_train_test)} proteins")
print(f"  - Val ∩ Test: {len(overlap_val_test)} proteins")

if len(overlap_train_val) == 0 and len(overlap_train_test) == 0 and len(overlap_val_test) == 0:
    print(f"\n✅ No overlaps detected! Splits are clean.")
else:
    print(f"\n⚠️  WARNING: Overlaps detected between splits!")


✓ Train split verification: 7989 proteins loaded
  Sample protein keys: ['accession', 'protein_name', 'gene_name', 'organism', 'sequence', 'length', 'function', 'go_terms', 'ec_numbers', 'function_words']

✓ Val split verification: 999 proteins loaded

✓ Test split verification: 999 proteins loaded

🔍 Checking for overlaps between splits:
  - Train ∩ Val: 0 proteins
  - Train ∩ Test: 0 proteins
  - Val ∩ Test: 0 proteins

✅ No overlaps detected! Splits are clean.
